In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist

In [2]:
num_classes = 10 # 0 to 9 digits
num_features = 784 # 28*28

learning_rate = 0.01
training_steps = 1000
batch_size = 256
display_step = 50

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])
x_train, x_test = x_train / 255., x_test / 255.

11493376/11490434 [==============================] - 60s 5us/step


In [4]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [14]:
W = tf.Variable(tf.ones([num_features, num_classes]), name="weight")
b = tf.Variable(tf.zeros([num_classes]), name="bias")

In [15]:
def logistic_regression(x):
    return tf.nn.softmax(tf.matmul(x, W) + b)

def cross_entropy(y_pred, y_true):
    y_true = tf.one_hot(y_true, depth=num_classes)
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))

def accuracy(y_pred, y_true):
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [16]:
optimizer = tf.optimizers.SGD(learning_rate)

In [17]:
def run_optimization(x, y):
    with tf.GradientTape() as g:
        pred = logistic_regression(x)
        loss = cross_entropy(pred, y)
    
    gradients = g.gradient(loss, [W,b])
    optimizer.apply_gradients(zip(gradients, [W,b]))

In [18]:
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    run_optimization(batch_x, batch_y)
    
    if step % display_step == 0:
        pred = logistic_regression(batch_x)
        loss = cross_entropy(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc)) 

step: 50, loss: 654.769348, accuracy: 0.792969
step: 100, loss: 735.912598, accuracy: 0.820312
step: 150, loss: 624.465027, accuracy: 0.847656
step: 200, loss: 557.240479, accuracy: 0.804688
step: 250, loss: 382.915253, accuracy: 0.886719
step: 300, loss: 443.499359, accuracy: 0.828125
step: 350, loss: 468.062744, accuracy: 0.812500
step: 400, loss: 687.035278, accuracy: 0.726562
step: 450, loss: 438.335480, accuracy: 0.890625
step: 500, loss: 627.526245, accuracy: 0.832031
step: 550, loss: 472.551147, accuracy: 0.875000
step: 600, loss: 482.123016, accuracy: 0.859375
step: 650, loss: 642.970642, accuracy: 0.800781
step: 700, loss: 700.598328, accuracy: 0.820312
step: 750, loss: 521.331665, accuracy: 0.847656
step: 800, loss: 604.353210, accuracy: 0.785156
step: 850, loss: 475.649445, accuracy: 0.839844
step: 900, loss: 687.757202, accuracy: 0.832031
step: 950, loss: 496.851898, accuracy: 0.863281
step: 1000, loss: 683.595764, accuracy: 0.777344
